# RNN 사용하기

쉽지는 않지만, RNN을 사용해보기로 하자.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('C:/Users/USER/Desktop/playground/chess_play/chess_rating_pred_dataset.csv')

앞에서 같은 등급의 체스 경기, 블리츠 300초 경기에 한정, 최대 70수 까지만 보고 레이팅을 맞히기로 결정하였다.
해당 데이터 셋을 바탕으로 예측을 해보자.

앞에서 했던 가중치나 유사도 기반의 문제점은
1) 순서에 대한 정보 무시
2) 체스 수라는 것은 상황에 따라 가중치가 다른데, 단순히 수가 많이 등장했다고 가중치를 부과할 수 없음

따라서 순서를 반영하는 RNN을 사용해야 한다. 여기서 수의 중요도보다는 수 자체를 사용해야 하므로, 벡터화 방법도 달라져야 한다.

## 학습 데이터 만들기, 벡터화 방법

우선 가장 큰 문제는 리스트가 지금 문자열로 저장되어 버렸다. 이를 해소하자

In [3]:
df['chess_string'] = df.chess_string.apply(lambda x: eval(x))

In [ ]:
df['chess_s']

그리고, word2vec을 해야 할 것이다.

In [4]:
from gensim.models.word2vec import Word2Vec

In [5]:
from sklearn.model_selection import train_test_split
X = df['chess_string']
y = df['grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42, stratify = y)

window는 4로 했다. 너- 나 - 너- 나 이렇게 4수를 보면 대응까지를 고려하므로 4수가 적당하다.

In [52]:
model = Word2Vec(sentences=X_train, size=10, window=4, min_count=0, sg=0)

In [53]:
model.wv.most_similar("e4")

[('d4', 0.9692720174789429),
 ('Nf3', 0.9569461941719055),
 ('Nc3', 0.9267320036888123),
 ('Bc4', 0.8996136784553528),
 ('d3', 0.8906040191650391),
 ('e5', 0.8897177577018738),
 ('Nf6', 0.8830223679542542),
 ('Nce7', 0.8805091977119446),
 ('d5', 0.8716991543769836),
 ('Nc6', 0.8695103526115417)]

상당히 유의미한 결과가보인다. 물론 완벽하진 않지만 e4의 가장 유사한 수로 d4와 Nf3를 뽑았다는 것만으로도, w2v의 위력이 보인다.

In [54]:
model['e4']

<ipython-input-54-b771cad6c9b1>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  model['e4']


array([ 1.6304363, -0.4004242, -2.859944 , -3.732466 , -2.814522 ,
       -3.173032 , -1.4030678, -1.0641822,  2.4807205, -1.9324874],
      dtype=float32)

In [67]:
model.wv.most_similar("O-O-O")

[('O-O', 0.957873523235321),
 ('Nbd7', 0.9423598051071167),
 ('Be7', 0.9304890632629395),
 ('Ne7', 0.9200345277786255),
 ('Ngf6', 0.9086468815803528),
 ('Bg5', 0.9033535122871399),
 ('Qd7', 0.8693109750747681),
 ('Nf6', 0.8683937191963196),
 ('Nd7', 0.8642125129699707),
 ('h3', 0.8633211255073547)]

캐슬링에 대해서도 캐슬링 길을열거나 킹사이드/퀸사이드의 유사성을 보여주고 있어서 학습이 잘 된 것처럼 보인다.

train에 없는 수가 등장할 수도 있는데, 이는 일관적으로 통일하자.

In [68]:
def get_word2vec_seq(x):
    answer_list = []
    for c in x:
        try:
            answer_list.append(model[c])
        except:
            answer_list.append([0.5] * 10)
    if len(answer_list) < 70:
        while len(answer_list) < 70:
            answer_list.append([0] * 10)
    return answer_list

In [71]:
X_train_w2v = X_train.apply(lambda x : get_word2vec_seq(x))
X_test_w2v = X_test.apply(lambda x : get_word2vec_seq(x))

<ipython-input-68-32b71a9b334d>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  answer_list.append(model[c])
